# Multi‑Head Attention — Teori

**Amaç:** Bu hücre seti, Transformer modellerinin merkezinde yer alan *Multi‑Head Attention* (Çok Başlıklı Dikkat) mekanizmasını adım adım teorik olarak açıklar. Matematiksel formülasyon, tensör şekilleri, sezgisel açıklama ve uygulamada dikkat edilmesi gereken noktalar yer alır.



## 1. Kısa Özet

* Attention, bir dizi öğe arasındaki bağımlılıkları öğrenmeyi sağlar; her öğe diğer öğelere "ne kadar dikkat etmeli" sorusuna yanıt arar.
* Scaled Dot‑Product Attention, Transformer'da kullanılan temel dikkat çeşididir.
* Multi‑Head Attention, aynı anda birden fazla dikkat "başı" çalıştırarak farklı alt‑uzaylarda ilişkileri yakalar.

## 2. Neden dikkat (Attention)?

Örnek: bir çeviri görevinde, hedefteki bir kelime kaynak cümlenin hangi kelimelerine dayanmalıdır? Dikkat mekanizması, hedef token için kaynak token'lara ağırlık (score) verir — böylece model bağlamdan dinamik olarak bilgi toplar.


## 3. Scaled Dot‑Product Attention (Ölçeklenmiş Nokta Çarpımı Dikkat)

Verilen sorgu (queries) $Q$, anahtarlar (keys) $K$ ve değerler (values) $V$ için:

[
\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^{T}}{\sqrt{d_k}}\right)V
]

* $Q \in \mathbb{R}^{n_q \times d_k}$, $K \in \mathbb{R}^{n_k \times d_k}$, $V \in \mathbb{R}^{n_k \times d_v}$
* Bölme faktörü $\sqrt{d_k}$, nokta çarpımı skorlarının varyansını kontrol ederek softmax'ten önce numerik kararlılık sağlar.
* Çıktı, her sorgu için değerlerin bir ağırlıklı toplamıdır.

**Adımlar:**

1. Skorları hesapla: $S = QK^{T}$ (boyut: $n_q \times n_k$)
2. Skorları ölçekle: $S' = S / \sqrt{d_k}$
3. Softmax uygula: $A = \text{softmax}(S')$ (satır bazlı softmax)
4. Değerleri topla: $O = A V$ (boyut: $n_q \times d_v$)

## 4. Multi‑Head Attention — Matematik

Multi‑Head Attention $h$ adet baş kullanır. Her başın kendi proje (lineer) matrisleri vardır:

* $W^{Q}*{i} \in \mathbb{R}^{d*{model} \times d_k}$
* $W^{K}*{i} \in \mathbb{R}^{d*{model} \times d_k}$
* $W^{V}*{i} \in \mathbb{R}^{d*{model} \times d_v}$

Her baş için:

[
\text{head}*i = \text{Attention}(QW^{Q}*{i}, KW^{K}*{i}, VW^{V}*{i})
]

Tüm başlar birleştirildikten sonra bir çıktı projeksiyonu uygulanır:

[
\text{MultiHead}(Q,K,V) = \text{Concat}(\text{head}_1,\dots,\text{head}_h)W^{O}
]

burada $W^{O} \in \mathbb{R}^{hd_v \times d_{model}}$.

**Tipik seçim:** $d_k = d_v = d_{model}/h$ iken her bir başın boyutu küçülür ve toplam hesap $d_{model}$'e yakın kalır.


## 5. Tensör Şekilleri (Batch içeren hâl)

* Girdi: $X$ şekli `(batch_size, seq_len, d_model)`
* Projeksiyonlar sonrası Q, K, V: `(batch_size, seq_len, h, d_k)` (veya `(batch_size, h, seq_len, d_k)` olarak yeniden şekillendirilir)
* Skor matrisi: `(batch_size, h, seq_len_q, seq_len_k)`
* Her başın çıktısı: `(batch_size, h, seq_len_q, d_v)`
* Birleştirilmiş çıktı: `(batch_size, seq_len_q, h * d_v)` → ardından `W^O` ile `(batch_size, seq_len_q, d_model)`


## 6. Sezgisel Anlatım

* Her baş, projeksiyonları sayesinde farklı bir "altyapı"ya (subspace) bakar — örneğin başlardan biri sözcük biçimsel eşleşmelere, diğeri sözdizimine, bir diğeri uzun menzilli ilişkilere odaklanabilir.
* Parçalamak, modelin eş zamanlı olarak farklı ilişki türlerini öğrenmesine izin verir.



## 7. Maskeler ve Kullanım Durumları

* **Padding Mask:** Düzensiz uzunluklarda dolgu (padding) tokenlarının dikkat alımını engellemek için.
* **Look‑Ahead / Causal Mask:** Otomatik dil modellemede, gelecekteki tokenların görülmesini engellemek için (decoder self‑attention).

Maskeler skorlara eklenir (büyük negatif sayılarla) softmax uygulamadan önce.



## 8. Hesaplama Karmaşıklığı

* Standart attention: $O(n^{2} \cdot d)$ zaman ve $O(n^{2})$ hafıza (n = seq_len)
* Bu sebeple uzun dizilerde dikkat pahalıdır; pek çok çalışma bu maliyeti azaltmaya yönelik alternatifler sunar (sparse, local, linear attention vb.).



## 9. Uygulama Notları / İpuçları

* `d_model` genelde `num_heads * d_k` ile eşleşmelidir.
* Kullandığınız başlangıç (initialization) ve normalizasyon (LayerNorm) sonuçları etkiler.
* Dropout, softmax sonrası attention ağırlıklarına uygulanır.
* Kullandığınız `dtype` (float32/float16) ve numerik kararlılık için ölçekleme önemlidir.



## 10. Kısa Örnek Sorular (Alıştırma)

1. Neden skoru $\sqrt{d_k}$ ile böleriz? (numerik stabilite ve gradient büyüklüğünü kontrol etme)
2. Eğer `d_model = 512` ve `num_heads = 8` ise `d_k` kaç olmalı? (cevap: 64)
3. Look‑ahead mask nasıl oluşturulur? (üçgen bir maskeler matrisi — üst üçgen `-inf`)



## 11. Kaynaklar / İleri Okuma

* Vaswani et al., *Attention Is All You Need* (2017)
* Transformer ders notları ve görselleştirme araçları



*Bu hücre teorik açıklamayı tamamlar. Sonraki hücrede bu matematiği PyTorch / NumPy ile çalışır bir `MultiHeadAttention` sınıfına dönüştüreceğiz.*
